Оригинальный скрипт. Даёт где-то 0.47 на тесте

In [3]:
## constants list
##
LOAD_TEST = False   ## do prediction for test
MAKE_SAMPLE = True  ## only do sample of 10k
FLAG_SAVE_MODEL_DATA = False

VAL_MISS_MARGIN = 10 ## min number of occurence for a brand or a name in data
NUM_BRANDS = 4500
NUM_CATEGORIES = 1250
NAME_MIN_DF = 10 ## lower bound for word count



%matplotlib inline
import matplotlib.pyplot as plt

import gc
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from time import gmtime, strftime

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from nltk.stem.porter import PorterStemmer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor


import scipy
import gc
import sys

#Add https://www.kaggle.com/anttip/wordbatch to your kernel Data Sources, 
#until Kaggle admins fix the wordbatch pip package installation
# sys.path.insert(0, '../input/wordbatch/wordbatch/')
##import wordbatch
##from wordbatch.extractors import WordBag, WordHash
##from wordbatch.models import FTRL, FM_FTRL

from nltk.corpus import stopwords
import re

def rmsle(y, y_pred):
     assert len(y) == len(y_pred)
     y = np.array(y)
     y_pred = np.array(y_pred)
     y_pred[y_pred<0] = 0
     y[y<0] = 0   
     
     return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_pred), 2)))


In [8]:
%%time
 
train = pd.read_table('input/train.tsv', engine='c')
train = train.drop(train[(train.price < 3.0)].index) ## prices lower than 3 are not allowed

if MAKE_SAMPLE:
    train = train.sample(n=100000).reset_index().drop('index',axis=1).copy(deep=True)


print('Train shape: ', train.shape)
merged = pd.concat([train])

nrow_train = train.shape[0]
del train


if LOAD_TEST:
    test = pd.read_table('input/test.tsv', engine='c')
    if MAKE_SAMPLE:
        test = test.sample(n=100000, random_state=201).reset_index().drop('index',axis=1).copy(deep=True)
    
    print('Test shape: ', test.shape)
    merged = pd.concat([merged, test])
    nrow_test = test.shape[0]
    del test
    
print('Merged shape: ', merged.shape)

merged.head()

Train shape:  (100000, 8)
Merged shape:  (100000, 8)
Wall time: 3.67 s


In [9]:
%%time

## Fill missing brand names
  
all_brands = set(merged['brand_name'].values)
merged.brand_name.fillna(value="missing", inplace=True)

# get to finding!
premissing = len(merged.loc[merged['brand_name'] == 'missing'])
def brandfinder(line):
    brand = line[0]
    name = line[1]
    namesplit = name.split(' ')
    if brand == 'missing':
        for x in namesplit:
            if x in all_brands:
                return name
    if name in all_brands:
        return name
    return brand

merged['brand_name'] = merged[['brand_name','name']].apply(brandfinder, axis = 1)

found = premissing-len(merged.loc[merged['brand_name'] == 'missing'])
print(found)

6278
Wall time: 4.47 s


In [10]:
%%time
## preprocess - splitting category names
##

tt = pd.DataFrame([ str(x).split("/") for x in merged['category_name']])
cat_col_names = ['general_cat','subcat_1','subcat_2','subcat_3','subcat_4']
tt.columns = cat_col_names

merged[cat_col_names] = tt
del tt

Wall time: 341 ms


In [11]:
%%time
## replace missing values

merged['category_name'].fillna(value='missing', inplace=True)
merged['general_cat'].fillna(value='missing', inplace=True)
merged['subcat_1'].fillna(value='missing', inplace=True)
merged['subcat_2'].fillna(value='missing', inplace=True)
merged['subcat_3'].fillna(value='missing', inplace=True)
merged['subcat_4'].fillna(value='missing', inplace=True)
merged['brand_name'].fillna(value='missing', inplace=True)
merged['item_description'].fillna('No description yet',inplace=True)
merged.item_description.replace('No description yet',"missing", inplace=True)


Wall time: 124 ms


In [28]:
%%time
### delete rare brands
##

pop_brand = merged['brand_name'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
merged.loc[merged['brand_name'].isin(pop_brand), 'mis_brand_name'] = 1
merged.loc[merged['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'

## delete rare categories
##

pop_category1 = merged['general_cat'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category2 = merged['subcat_1'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category3 = merged['subcat_2'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
# merged.loc[merged['general_cat'].isin(pop_brand), 'mis_general_cat'] = 1
# merged.loc[merged['general_cat'].isin(pop_category1), 'general_cat'] = 'missing'
# merged.loc[merged['subcat_1'].isin(pop_brand), 'mis_subcat_1'] = 1
# merged.loc[merged['subcat_1'].isin(pop_category2), 'subcat_1'] = 'missing'
# merged.loc[merged['subcat_2'].isin(pop_brand), 'mis_subcat_2'] = 1
# merged.loc[merged['subcat_2'].isin(pop_category3), 'subcat_2'] = 'missing'
    
print (pop_brand)
print (len(pop_category1),pop_category1[1:5])
print (len(pop_category2),pop_category2[1:5])
print (len(pop_category3),pop_category3[1:5])


['Wilsons Leather' 'Bic' 'FULL TILT' ..., 'Philips Christmas lights'
 'Jamberry tru shine kit' 'LuLaRoe XXS Floral Perfect T, BNWT']
0 []
11 ['Crochet' 'Knitting' 'Woodworking' 'Candles']
282 ['Shrug' 'Wool' 'Tshirt' 'Motorcycle & Powersports']
Wall time: 777 ms


In [29]:
%%time
## count non-utf characters
s = "☆Please read bio☆ Women's sandal bundle"
def count_nonutf(s):
    s_clean = re.sub(r'[^\x00-\x7F]+','', s)
    return len(s)-len(s_clean)


tt = pd.DataFrame([ count_nonutf(x) for x in merged['item_description']])
merged['non_utf_descr'] = tt

tt = pd.DataFrame([ count_nonutf(x) for x in merged['name']])
merged['non_utf_name'] = tt



Wall time: 1.39 s


In [32]:
%%time

## Building features with counts of all Non-utf symbols
##
s = "☆Please read bio☆ Women's sandal bundle"

def all_non_utf(s, string_pattern):
    s_clean = re.sub(string_pattern,'', s)
    return list((s_clean))

print ("Counting non utf symbols")

tkn_nonutf = TfidfVectorizer(max_features = 500, 
                ngram_range = (1,1),
               tokenizer = lambda x: all_non_utf(x,r'[\x00-\x7F]'))
X_non_utf_uniq =tkn_nonutf.fit_transform(merged['item_description'])

tkn_punkt = TfidfVectorizer(max_features = 500, 
                ngram_range = (1,1),
               tokenizer = lambda x: all_non_utf(x,r'[^\.!{}\,[\]?:\'\"*~@]'))
X_punkt =tkn_nonutf.fit_transform(merged['item_description'])


Counting non utf symbols
Wall time: 9.79 s


In [33]:
%%time
## Count number of letters in upper case
##
def perc_symbols(s,pattern):
    try:
        s_clean = re.sub(pattern,'', s)
        return len(s_clean)/len(s)
    except:
        return 0

def count_symbols(s, pattern):
    try:
        s_clean = re.sub(pattern,'', s)
        return len(s_clean)
    except:
        return 0
    
## upper case
##

tt = pd.DataFrame([perc_symbols(x,r'[^A-Z ]+') for x in merged['item_description']])
merged['upper_perc_descr'] = tt

tt = pd.DataFrame([perc_symbols(x,r'[^A-Z ]+') for x in merged['name']])
merged['upper_perc_name'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^A-Z ]+') for x in merged['item_description']])
merged['count_upper_descr'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^A-Z ]+') for x in merged['name']])
merged['count_upper_name'] = tt

## exclamation points
tt = pd.DataFrame([perc_symbols(x,r'[^!]+') for x in merged['item_description']])
merged['upper_perc_excl'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^!]+') for x in merged['item_description']])
merged['count_upper_excl'] = tt



Wall time: 3.93 s


In [34]:
%%time

## length of description and brand name

def wordCount(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0
    
merged['desc_len'] = merged['item_description'].apply(lambda x: wordCount(x))
merged['name_len'] = merged['name'].apply(lambda x: wordCount(x))

Wall time: 799 ms


In [35]:
merged.loc[1,'item_description']

"This is a bundled listing for Bunker Hill Security Door/Window Entry Alarms. There is 3 packs of them. One pack of one, one pack of two, and one pack of 3. Brand new, double-sided tape is included. Batteries are included, but you may still need AAA's. Instructions are also included."

In [36]:
def search_in_str(s,string_val):
    instr = re.findall(string_val,s,flags=re.IGNORECASE)
    if len(instr)>0:
        return 1
    else:
        return 0

def search_in_merged(string_val):
    tt1 = merged['item_description'].apply(lambda x: search_in_str(x,string_val))
    tt2 = merged['name'].apply(lambda x: search_in_str(x,string_val))

    tt =  pd.DataFrame({'tt1':tt1,'tt2':tt2}).max(axis=1)
    return tt

start = time.clock()
merged['a_free'] = search_in_merged('Free')
merged['a_free_ship'] = search_in_merged('Free ship')
merged['a_new'] = search_in_merged('new')
merged['a_sale'] = search_in_merged('sale')
merged['a_gift'] = search_in_merged('gift')
merged['a_nwt'] = search_in_merged('nwt')
merged['a_100perc'] = search_in_merged('100%')
print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 0.09326178991580415


In [37]:
%%time
MAX_FEAT_DESCP = 50000

print("Name Encodings")


## count = CountVectorizer(min_df=NAME_MIN_DF)
##  X_name = count.fit_transform(merged["name"])
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, 
                              ngram_range = (1,3),
                              stop_words = "english")
## X_name = count_descp.fit_transform(merged["name"])


count =  CountVectorizer(
        ngram_range=(1, 2),
        max_features= 50000)
X_name = count.fit_transform(merged["name"])

print("Category Encoders")
count_category = CountVectorizer()
X_category = count_category.fit_transform(merged["category_name"])
X_gen_cat = count_category.fit_transform(merged["general_cat"])
X_cat1 = count_category.fit_transform(merged["subcat_1"])
X_cat2 = count_category.fit_transform(merged["subcat_2"])
X_cat3 = count_category.fit_transform(merged["subcat_3"])


print("Descp encoders")
count_descp = TfidfVectorizer(max_features = 100000, 
                              ngram_range = (1,3),
                              stop_words = "english")
X_descp = count_descp.fit_transform(merged["item_description"])

print("Brand encoders")
vect_brand = LabelBinarizer(sparse_output=True)
X_brand = vect_brand.fit_transform(merged["brand_name"])

print("Dummy Encoders")
merged['item_condition_id'] = merged['item_condition_id'].astype('category')
merged['shipping'] = merged['shipping'].astype('category')
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(merged[["item_condition_id", "shipping"]], sparse = True).values)

merged['desc_len'] = merged['desc_len'].astype('category')
merged['name_len'] = merged['name_len'].astype('category')
X_dummies2 = scipy.sparse.csr_matrix(pd.get_dummies(merged[["desc_len", "name_len"]], sparse = True).values)


Name Encodings
Category Encoders
Descp encoders
Brand encoders
Dummy Encoders
Wall time: 33.4 s


In [42]:
X = scipy.sparse.hstack((X_dummies 
                          ,X_descp
                          ,X_brand
                          ,X_gen_cat, X_cat1, X_cat2, X_cat3
                          ,X_name
                          ,X_punkt
#                         ,merged[['mis_general_cat','mis_subcat_1','mis_subcat_2','mis_subcat_3','mis_subcat_4','mis_brand_name','mis_item_description']]
#                        ,merged[['non_utf_descr','non_utf_name','upper_perc_descr','upper_perc_name']]
                         ,X_non_utf_uniq
#                        ,merged[['a_free','a_free_ship','a_new','a_sale','a_gift','a_nwt','a_100perc']]
                        ,merged[['count_upper_excl']]
#                       ,merged[['desc_len','name_len']]
                        ,X_dummies2
                        )).tocsr()
print (X.shape)
## del X_dummies, X_descp, X_brand,X_category,X_name, X_gen_cat, X_cat1, X_cat2, X_cat3
gc.collect()

## 'count_upper_descr','count_upper_name','len_descr','len_name'

(100000, 151492)


24

In [39]:
## save data
##

if FLAG_SAVE_MODEL_DATA:
    scipy.sparse.save_npz('model/x.npz',X)
    merged.to_csv('model/merged.csv',encoding='utf-8')

In [40]:
## load data
##

if FLAG_SAVE_MODEL_DATA:
    X = scipy.sparse.load_npz('model/x.npz')
    merged = pd.read_csv('model/merged.csv')


In [41]:
%%time
## https://www.kaggle.com/apapiu/ridge-script
##

## Split into test and train
##
y_merged_train = merged['price'][:nrow_train]
y_merged_train = np.log1p(y_merged_train)
X_merged_train = X[:nrow_train]

X_train, X_valid, y_train, y_valid = train_test_split(X_merged_train, y_merged_train, test_size=0.2, random_state =102)
k_fold = KFold(n_splits=5,random_state=101)

print (X_train.shape, len(y_train))
print (X_valid.shape, len(y_valid))


scorer_func = make_scorer(mean_squared_error,greater_is_better=False)


mm = Ridge(solver = "auto",  alpha=1.0,
           normalize=False, fit_intercept=True, max_iter=100, random_state=120)

# scores = cross_val_score(mm, X_train, y_train, 
#                          cv=k_fold, n_jobs=-1,scoring = scorer_func)

# scores = np.sqrt(-scores)
# print (np.mean(scores),scores)

mm.fit(X_train,y_train)

## test on validation
mm.fit(X_train, y_train)
y_pred = mm.predict(X_valid)
y_pred[y_pred<1.386]=1.386

print ("")
print ("RMSLE: ",rmsle(np.expm1(y_valid),np.expm1(y_pred)))
print ("RMSE: ",mean_squared_error(y_valid,y_pred))
print ("R2: ",r2_score(y_valid,y_pred))

(80000, 151492) 80000
(20000, 151492) 20000


C:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



RMSLE:  0.506808047441
RMSE:  0.256854396951
R2:  0.542144952269
Wall time: 38.2 s


0.461



In [31]:
%%time
X_train, X_valid, y_train, y_valid = train_test_split(X_merged_train[:,~(mm.coef_==0)], y_merged_train, test_size=0.2, random_state =102)

X_train, X_watch, y_train, y_watch = train_test_split(X_train, y_train, test_size=0.2, random_state =101)
print (X_train.shape,X_watch.shape,X_valid.shape)

params2 = {
        'learning_rate': 0.2,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 120,
        'verbosity': -1,
        'metric': 'RMSE',
        'data_random_seed': 2,
        'bagging_fraction': 0.7,
        'nthread': 4
}

d_train = lgb.Dataset(X_train, label=y_train)
d_watch = lgb.Dataset(X_watch, label=y_watch)


watchlist2 = [d_watch]    
lgb_model = lgb.train(params2, train_set=d_train, num_boost_round=8000, valid_sets=watchlist2, \
    early_stopping_rounds=250, verbose_eval=True)     

y_pred = lgb_model.predict(X_valid)
y_pred[y_pred<1.386]=1.386

print ("RMSLE: ",rmsle(np.expm1(y_valid),np.expm1(y_pred)))
print ("RMSE: ",mean_squared_error(y_valid,y_pred))
print ("R2: ",r2_score(y_valid,y_pred))


(64000, 100045) (16000, 100045) (20000, 100045)
[1]	valid_0's rmse: 0.729347
Training until validation scores don't improve for 250 rounds.
[2]	valid_0's rmse: 0.717353
[3]	valid_0's rmse: 0.707653
[4]	valid_0's rmse: 0.699816
[5]	valid_0's rmse: 0.692166
[6]	valid_0's rmse: 0.68622
[7]	valid_0's rmse: 0.681802
[8]	valid_0's rmse: 0.676683
[9]	valid_0's rmse: 0.672824
[10]	valid_0's rmse: 0.669676
[11]	valid_0's rmse: 0.665565
[12]	valid_0's rmse: 0.66311
[13]	valid_0's rmse: 0.660303
[14]	valid_0's rmse: 0.657903
[15]	valid_0's rmse: 0.655563
[16]	valid_0's rmse: 0.652789
[17]	valid_0's rmse: 0.650411
[18]	valid_0's rmse: 0.648194
[19]	valid_0's rmse: 0.646695
[20]	valid_0's rmse: 0.64491
[21]	valid_0's rmse: 0.643535
[22]	valid_0's rmse: 0.64188
[23]	valid_0's rmse: 0.640487
[24]	valid_0's rmse: 0.639096
[25]	valid_0's rmse: 0.637539
[26]	valid_0's rmse: 0.636003
[27]	valid_0's rmse: 0.634351
[28]	valid_0's rmse: 0.633349
[29]	valid_0's rmse: 0.632218
[30]	valid_0's rmse: 0.631123
[3

[268]	valid_0's rmse: 0.550954
[269]	valid_0's rmse: 0.550825
[270]	valid_0's rmse: 0.550709
[271]	valid_0's rmse: 0.550604
[272]	valid_0's rmse: 0.550558
[273]	valid_0's rmse: 0.550417
[274]	valid_0's rmse: 0.550279
[275]	valid_0's rmse: 0.550164
[276]	valid_0's rmse: 0.550051
[277]	valid_0's rmse: 0.5499
[278]	valid_0's rmse: 0.549823
[279]	valid_0's rmse: 0.549815
[280]	valid_0's rmse: 0.549688
[281]	valid_0's rmse: 0.549543
[282]	valid_0's rmse: 0.549399
[283]	valid_0's rmse: 0.549309
[284]	valid_0's rmse: 0.549212
[285]	valid_0's rmse: 0.549095
[286]	valid_0's rmse: 0.549043
[287]	valid_0's rmse: 0.548907
[288]	valid_0's rmse: 0.548782
[289]	valid_0's rmse: 0.5487
[290]	valid_0's rmse: 0.548637
[291]	valid_0's rmse: 0.548553
[292]	valid_0's rmse: 0.548478
[293]	valid_0's rmse: 0.548406
[294]	valid_0's rmse: 0.548325
[295]	valid_0's rmse: 0.548323
[296]	valid_0's rmse: 0.548234
[297]	valid_0's rmse: 0.548092
[298]	valid_0's rmse: 0.547961
[299]	valid_0's rmse: 0.547902
[300]	valid_

[538]	valid_0's rmse: 0.531532
[539]	valid_0's rmse: 0.531473
[540]	valid_0's rmse: 0.531429
[541]	valid_0's rmse: 0.531372
[542]	valid_0's rmse: 0.531299
[543]	valid_0's rmse: 0.531362
[544]	valid_0's rmse: 0.5313
[545]	valid_0's rmse: 0.531174
[546]	valid_0's rmse: 0.531144
[547]	valid_0's rmse: 0.531092
[548]	valid_0's rmse: 0.531081
[549]	valid_0's rmse: 0.531004
[550]	valid_0's rmse: 0.531001
[551]	valid_0's rmse: 0.530991
[552]	valid_0's rmse: 0.530952
[553]	valid_0's rmse: 0.530869
[554]	valid_0's rmse: 0.530813
[555]	valid_0's rmse: 0.530777
[556]	valid_0's rmse: 0.530716
[557]	valid_0's rmse: 0.530605
[558]	valid_0's rmse: 0.530575
[559]	valid_0's rmse: 0.530598
[560]	valid_0's rmse: 0.530569
[561]	valid_0's rmse: 0.530544
[562]	valid_0's rmse: 0.530532
[563]	valid_0's rmse: 0.530511
[564]	valid_0's rmse: 0.530433
[565]	valid_0's rmse: 0.530374
[566]	valid_0's rmse: 0.530324
[567]	valid_0's rmse: 0.530246
[568]	valid_0's rmse: 0.530262
[569]	valid_0's rmse: 0.530242
[570]	vali

[803]	valid_0's rmse: 0.523627
[804]	valid_0's rmse: 0.523609
[805]	valid_0's rmse: 0.523562
[806]	valid_0's rmse: 0.523564
[807]	valid_0's rmse: 0.523545
[808]	valid_0's rmse: 0.523515
[809]	valid_0's rmse: 0.523513
[810]	valid_0's rmse: 0.523473
[811]	valid_0's rmse: 0.523458
[812]	valid_0's rmse: 0.523463
[813]	valid_0's rmse: 0.523424
[814]	valid_0's rmse: 0.523404
[815]	valid_0's rmse: 0.523411
[816]	valid_0's rmse: 0.523404
[817]	valid_0's rmse: 0.523371
[818]	valid_0's rmse: 0.523329
[819]	valid_0's rmse: 0.523329
[820]	valid_0's rmse: 0.523329
[821]	valid_0's rmse: 0.52332
[822]	valid_0's rmse: 0.523256
[823]	valid_0's rmse: 0.52318
[824]	valid_0's rmse: 0.523187
[825]	valid_0's rmse: 0.52314
[826]	valid_0's rmse: 0.523126
[827]	valid_0's rmse: 0.523166
[828]	valid_0's rmse: 0.523129
[829]	valid_0's rmse: 0.523097
[830]	valid_0's rmse: 0.523053
[831]	valid_0's rmse: 0.523036
[832]	valid_0's rmse: 0.523018
[833]	valid_0's rmse: 0.523008
[834]	valid_0's rmse: 0.523028
[835]	valid

[1069]	valid_0's rmse: 0.519168
[1070]	valid_0's rmse: 0.51914
[1071]	valid_0's rmse: 0.519146
[1072]	valid_0's rmse: 0.519119
[1073]	valid_0's rmse: 0.51912
[1074]	valid_0's rmse: 0.519101
[1075]	valid_0's rmse: 0.519101
[1076]	valid_0's rmse: 0.51911
[1077]	valid_0's rmse: 0.519115
[1078]	valid_0's rmse: 0.519102
[1079]	valid_0's rmse: 0.519056
[1080]	valid_0's rmse: 0.519033
[1081]	valid_0's rmse: 0.519028
[1082]	valid_0's rmse: 0.518994
[1083]	valid_0's rmse: 0.518967
[1084]	valid_0's rmse: 0.518962
[1085]	valid_0's rmse: 0.518955
[1086]	valid_0's rmse: 0.518976
[1087]	valid_0's rmse: 0.518937
[1088]	valid_0's rmse: 0.518834
[1089]	valid_0's rmse: 0.518688
[1090]	valid_0's rmse: 0.518658
[1091]	valid_0's rmse: 0.518421
[1092]	valid_0's rmse: 0.518395
[1093]	valid_0's rmse: 0.5183
[1094]	valid_0's rmse: 0.518251
[1095]	valid_0's rmse: 0.518251
[1096]	valid_0's rmse: 0.518232
[1097]	valid_0's rmse: 0.518218
[1098]	valid_0's rmse: 0.518192
[1099]	valid_0's rmse: 0.518164
[1100]	valid_

...

[2629]	valid_0's rmse: 0.508112
[2630]	valid_0's rmse: 0.508103
[2631]	valid_0's rmse: 0.508105
[2632]	valid_0's rmse: 0.508097
[2633]	valid_0's rmse: 0.5081
[2634]	valid_0's rmse: 0.508115
[2635]	valid_0's rmse: 0.508123
[2636]	valid_0's rmse: 0.508122
[2637]	valid_0's rmse: 0.50811
[2638]	valid_0's rmse: 0.508097
[2639]	valid_0's rmse: 0.508107
[2640]	valid_0's rmse: 0.508135
[2641]	valid_0's rmse: 0.508139
[2642]	valid_0's rmse: 0.508118
[2643]	valid_0's rmse: 0.508123
[2644]	valid_0's rmse: 0.508122
[2645]	valid_0's rmse: 0.5081
[2646]	valid_0's rmse: 0.508087
[2647]	valid_0's rmse: 0.508106
[2648]	valid_0's rmse: 0.508125
[2649]	valid_0's rmse: 0.50814
[2650]	valid_0's rmse: 0.50813
[2651]	valid_0's rmse: 0.50816
[2652]	valid_0's rmse: 0.508172
[2653]	valid_0's rmse: 0.50818
[2654]	valid_0's rmse: 0.508163
[2655]	valid_0's rmse: 0.508164
[2656]	valid_0's rmse: 0.508165
[2657]	valid_0's rmse: 0.50815
[2658]	valid_0's rmse: 0.5081
[2659]	valid_0's rmse: 0.508069
[2660]	valid_0's rms

[2890]	valid_0's rmse: 0.507759
[2891]	valid_0's rmse: 0.507751
[2892]	valid_0's rmse: 0.507746
[2893]	valid_0's rmse: 0.507752
[2894]	valid_0's rmse: 0.507738
[2895]	valid_0's rmse: 0.507749
[2896]	valid_0's rmse: 0.507751
[2897]	valid_0's rmse: 0.507751
[2898]	valid_0's rmse: 0.507754
[2899]	valid_0's rmse: 0.507757
[2900]	valid_0's rmse: 0.507762
[2901]	valid_0's rmse: 0.507761
[2902]	valid_0's rmse: 0.507773
[2903]	valid_0's rmse: 0.507755
[2904]	valid_0's rmse: 0.507736
[2905]	valid_0's rmse: 0.507735
[2906]	valid_0's rmse: 0.507719
[2907]	valid_0's rmse: 0.507696
[2908]	valid_0's rmse: 0.507719
[2909]	valid_0's rmse: 0.507719
[2910]	valid_0's rmse: 0.507701
[2911]	valid_0's rmse: 0.507687
[2912]	valid_0's rmse: 0.507608
[2913]	valid_0's rmse: 0.507587
[2914]	valid_0's rmse: 0.507585
[2915]	valid_0's rmse: 0.507595
[2916]	valid_0's rmse: 0.507591
[2917]	valid_0's rmse: 0.507596
[2918]	valid_0's rmse: 0.507595
[2919]	valid_0's rmse: 0.507577
[2920]	valid_0's rmse: 0.507562
[2921]	v

[3153]	valid_0's rmse: 0.507466
[3154]	valid_0's rmse: 0.507468
[3155]	valid_0's rmse: 0.507463
[3156]	valid_0's rmse: 0.507459
[3157]	valid_0's rmse: 0.507459
[3158]	valid_0's rmse: 0.507487
[3159]	valid_0's rmse: 0.507489
[3160]	valid_0's rmse: 0.507502
[3161]	valid_0's rmse: 0.507506
[3162]	valid_0's rmse: 0.507503
[3163]	valid_0's rmse: 0.507498
[3164]	valid_0's rmse: 0.507506
[3165]	valid_0's rmse: 0.507508
[3166]	valid_0's rmse: 0.50749
[3167]	valid_0's rmse: 0.507498
[3168]	valid_0's rmse: 0.507517
[3169]	valid_0's rmse: 0.507513
[3170]	valid_0's rmse: 0.507514
[3171]	valid_0's rmse: 0.507521
[3172]	valid_0's rmse: 0.507529
[3173]	valid_0's rmse: 0.507528
[3174]	valid_0's rmse: 0.507505
[3175]	valid_0's rmse: 0.507492
[3176]	valid_0's rmse: 0.507496
[3177]	valid_0's rmse: 0.507496
[3178]	valid_0's rmse: 0.507501
[3179]	valid_0's rmse: 0.507501
[3180]	valid_0's rmse: 0.507497
[3181]	valid_0's rmse: 0.50749
[3182]	valid_0's rmse: 0.507496
[3183]	valid_0's rmse: 0.507494
[3184]	val

[3414]	valid_0's rmse: 0.507452
[3415]	valid_0's rmse: 0.507451
[3416]	valid_0's rmse: 0.507459
[3417]	valid_0's rmse: 0.507461
[3418]	valid_0's rmse: 0.507446
[3419]	valid_0's rmse: 0.507463
[3420]	valid_0's rmse: 0.507465
[3421]	valid_0's rmse: 0.507463
[3422]	valid_0's rmse: 0.50747
[3423]	valid_0's rmse: 0.507467
[3424]	valid_0's rmse: 0.507461
[3425]	valid_0's rmse: 0.507441
[3426]	valid_0's rmse: 0.507433
[3427]	valid_0's rmse: 0.507435
[3428]	valid_0's rmse: 0.507431
[3429]	valid_0's rmse: 0.507422
[3430]	valid_0's rmse: 0.507412
[3431]	valid_0's rmse: 0.50742
[3432]	valid_0's rmse: 0.507418
[3433]	valid_0's rmse: 0.50738
[3434]	valid_0's rmse: 0.507384
[3435]	valid_0's rmse: 0.507384
[3436]	valid_0's rmse: 0.507382
[3437]	valid_0's rmse: 0.507395
[3438]	valid_0's rmse: 0.507388
[3439]	valid_0's rmse: 0.507384
[3440]	valid_0's rmse: 0.507381
[3441]	valid_0's rmse: 0.507372
[3442]	valid_0's rmse: 0.507371
[3443]	valid_0's rmse: 0.50737
[3444]	valid_0's rmse: 0.507362
[3445]	valid

[3675]	valid_0's rmse: 0.507482
[3676]	valid_0's rmse: 0.507489
[3677]	valid_0's rmse: 0.507496
[3678]	valid_0's rmse: 0.507506
[3679]	valid_0's rmse: 0.507495
[3680]	valid_0's rmse: 0.507505
[3681]	valid_0's rmse: 0.507512
[3682]	valid_0's rmse: 0.507509
[3683]	valid_0's rmse: 0.507511
[3684]	valid_0's rmse: 0.507521
[3685]	valid_0's rmse: 0.507499
[3686]	valid_0's rmse: 0.507493
[3687]	valid_0's rmse: 0.507492
[3688]	valid_0's rmse: 0.507484
[3689]	valid_0's rmse: 0.507464
[3690]	valid_0's rmse: 0.507473
[3691]	valid_0's rmse: 0.507475
[3692]	valid_0's rmse: 0.507469
[3693]	valid_0's rmse: 0.50749
[3694]	valid_0's rmse: 0.507485
[3695]	valid_0's rmse: 0.507481
[3696]	valid_0's rmse: 0.507487
[3697]	valid_0's rmse: 0.507493
Early stopping, best iteration is:
[3447]	valid_0's rmse: 0.507308
RMSLE:  0.515938642835
RMSE:  0.26619268317
R2:  0.519464934135
Wall time: 2min 20s


In [ ]:
X_test = X[nrow_train:]

y_pred = lgb_model.predict(X_test)

y_pred[y_pred<1.386]=1.386
y_pred = np.expm1(y_pred)

sub = pd.DataFrame()
sub['test_id'] = merged['test_id'][nrow_train:].astype(int)
sub['price'] = y_pred
sub.to_csv('result.csv', index=False)
print ("fini")


In [ ]:
## https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling
##